<a href="https://colab.research.google.com/github/MeenTers/Project/blob/main/Project_SI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch
from torch import nn, load, utils
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, transforms, models
from torchvision.models.vgg import model_urls
from os import path, listdir
model_urls['vgg19'] = model_urls['vgg19'].replace('https://', 'http://')

In [3]:
from scipy.spatial import distance_matrix
from scipy.spatial import distance

In [4]:
import pandas as pd

# Prepearing the model

In [5]:
class VGG:
	def __init__(self):
		model = models.vgg19(pretrained=True, progress=True)
		model.classifier = nn.Sequential(*list(model.classifier.children())[:3])
		self.model = model.cuda().eval()

	def __call__(self, x):
		return self.model(x)

In [6]:
vgg = VGG()

# Features Extraction

In [7]:
def get_features(model, loader):
    features = []
    with torch.no_grad():
        for batch, _ in tqdm(loader):
            if torch.cuda.is_available():
                batch = batch.cuda()
            b_features = model(batch).detach().cpu().numpy()
            for f in b_features:
                features.append(f)

    return features

# Loading the data

In [8]:
def get_dataset(images_path):
  transform = transforms.Compose([
    transforms.Resize(size=32),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
  ])

  dataset = datasets.ImageFolder(images_path, transform=transform)
  loader = utils.data.DataLoader(dataset, batch_size=100, shuffle=False, num_workers=1, pin_memory=True)
  return dataset, loader

# Test multi-view image

In [9]:
car_dataset , car_loader = get_dataset(r'/content/drive/MyDrive/Project_si/car_model/')

In [10]:
damage_dataset , damage_loader = get_dataset(r'/content/drive/MyDrive/Project_si/model_damage/')

In [11]:
car_base = get_features(model=vgg,loader=car_loader)

100%|██████████| 1/1 [00:00<00:00,  4.59it/s]


In [12]:
len(car_base[0])

4096

In [13]:
car_base[0]

array([0.      , 0.      , 3.960284, ..., 0.      , 0.      , 0.      ],
      dtype=float32)

In [14]:
car_damage = get_features(model=vgg,loader=damage_loader)

100%|██████████| 1/1 [00:00<00:00,  5.43it/s]


In [15]:
len(car_damage[0])

4096

# Distance

In [16]:
car_image = '/content/drive/MyDrive/Project_si/car_model/ID1'
damage_image = '/content/drive/MyDrive/Project_si/model_damage/ID1'

In [17]:
distance.euclidean(car_base[1], car_damage[1])

13.258048057556152

# compute distance


In [18]:
def get_euclidean(basecar,diff_view):
    A = []
    for i in range(len(diff_view)):
        x = distance.euclidean(basecar[0],diff_view[i])
        A.append(x)
    return(A)

# ID1

In [19]:
ID1B_dataset, ID1B_loader = get_dataset(r'/content/drive/MyDrive/Project_si/Test_Dist/Base_car/ID1')
ID1_dataset,ID1_loader    = get_dataset(r'/content/drive/MyDrive/Project_si/Test_Dist/different_view/ID1')

In [20]:
base_id1 = get_features(vgg,ID1B_loader)
car_id1  = get_features(vgg,ID1_loader)

100%|██████████| 1/1 [00:00<00:00,  5.81it/s]


In [21]:
id1 = get_euclidean(base_id1,car_id1)
id1l = ['id1F','id1F','id1F','id1F']
id1l1 = ['id1F1','id1F2','id1F3','id2F']

In [22]:
dict1 = {'Model1':id1l, 'Model2':id1l1, 'Distance':id1}

# เปรียบเทียบระยะรถ ID1

In [23]:
df1 = pd.DataFrame(dict1)
df1

,Model1,Model2,Distance
0,id1F,id1F1,28.068737
1,id1F,id1F2,36.003323
2,id1F,id1F3,32.046677
3,id1F,id2F,117.247307


# ID2


In [24]:
ID2B_dataset, ID2B_loader = get_dataset(r'/content/drive/MyDrive/Project_si/Test_Dist/Base_car/ID2')
ID2_dataset,ID2_loader    = get_dataset(r'/content/drive/MyDrive/Project_si/Test_Dist/different_view/ID2')

In [25]:
base_id2 = get_features(vgg,ID2B_loader)
car_id2  = get_features(vgg,ID2_loader)

100%|██████████| 1/1 [00:00<00:00,  5.59it/s]


In [26]:
id2 = get_euclidean(base_id2,car_id2)
id2l = ['id2F','id2F','id2F','id2F']
id2l1 = ['id1F','id2F1','id2F2','id2F3']

In [27]:
dict2 = {'Model1':id2l, 'Model2':id2l1, 'Distance':id2}

In [28]:
df2 = pd.DataFrame(dict2)
df2

,Model1,Model2,Distance
0,id2F,id1F,117.247307
1,id2F,id2F1,40.263817
2,id2F,id2F2,35.501324
3,id2F,id2F3,60.388138


# ID3

In [29]:
ID3B_dataset, ID3B_loader = get_dataset(r'/content/drive/MyDrive/Project_si/Test_Dist/Base_car/ID3')
ID3_dataset,ID3_loader    = get_dataset(r'/content/drive/MyDrive/Project_si/Test_Dist/different_view/ID3')

In [30]:
base_id3 = get_features(vgg,ID3B_loader)
car_id3  = get_features(vgg,ID3_loader)

100%|██████████| 1/1 [00:00<00:00,  5.67it/s]


In [31]:
id3 = get_euclidean(base_id3,car_id3)
id3l = ['id3F','id3F','id3F','id3F']
id3l1 = ['id2F','id3F1','id3F2','id3F3']

In [32]:
dict3 = {'Model1':id3l, 'Model2':id3l1, 'Distance':id3}

In [33]:
df3 = pd.DataFrame(dict3)
df3

,Model1,Model2,Distance
0,id3F,id2F,65.138908
1,id3F,id3F1,23.942015
2,id3F,id3F2,54.359196
3,id3F,id3F3,81.706596


# ID4

In [34]:
ID4B_dataset, ID4B_loader = get_dataset(r'/content/drive/MyDrive/Project_si/Test_Dist/Base_car/ID4')
ID4_dataset,ID4_loader    = get_dataset(r'/content/drive/MyDrive/Project_si/Test_Dist/different_view/ID4')

In [35]:
base_id4 = get_features(vgg,ID4B_loader)
car_id4  = get_features(vgg,ID4_loader)

100%|██████████| 1/1 [00:00<00:00,  5.68it/s]


In [36]:
id4 = get_euclidean(base_id4,car_id4)
id4l = ['id4F','id4F','id4F','id4F']
id4l1 = ['id3F','id4F1','id4F2','id4F3']

In [37]:
dict4 = {'Model1':id4l, 'Model2':id4l1, 'Distance':id4}

In [38]:
df4 = pd.DataFrame(dict4)
df4

,Model1,Model2,Distance
0,id4F,id3F,36.646328
1,id4F,id4F1,18.773842
2,id4F,id4F2,46.188713
3,id4F,id4F3,53.507984


# รวมตาราง

In [39]:
total_df = pd.concat([df1,df2,df3,df4])
total_df

,Model1,Model2,Distance
0,id1F,id1F1,28.068737
1,id1F,id1F2,36.003323
2,id1F,id1F3,32.046677
3,id1F,id2F,117.247307
0,id2F,id1F,117.247307
1,id2F,id2F1,40.263817
2,id2F,id2F2,35.501324
3,id2F,id2F3,60.388138
0,id3F,id2F,65.138908
1,id3F,id3F1,23.942015
